In [1]:
import os
import tensorflow as tf
import time
import datetime
import numpy as np
from mlp.data_providers import MNISTDataProvider, AugmentedCIFAR10DataProvider, AugmentedCIFAR100DataProvider, CIFAR100DataProvider, CIFAR10DataProvider
from mlp.tf_layers import FCLayer, ConvLayer, max_pool_2x2
from mlp.image_transforms import random_flip, random_crop, center_crop, random_flip_small
from mlp.GAN_models import GAN, WasserstienGAN, CWGAN, MTLGAN
import mlp.tf_utils as utils
import matplotlib.pyplot as plt
import seaborn as sns
import cPickle
%matplotlib inline

# Seed a random number generator
seed = 24102016 
rng = np.random.RandomState(seed)

In [2]:
model_name = 'MTL_GAN'
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [3]:
batch_size = 64  # batch size for training
z_dim = 100  # size of input vector to generator has to include dim of y
gen_learning_rate = 2e-4
disc_learning_rate = 2e-4
optimizer_param = 0.9  # beta1 for Adam optimizer / decay for RMSProp
max_itrs = 1000  # No. of iterations to train model
main_image_size = 32  # Size of actual images, Size of images to be generated at.
aux_image_size = 32  # Size of actual images, Size of images to be generated at.
optimizer = "Adam"  # Optimizer to use for training
gen_dim = 16  # dimension of first layer in generator
logs_dir = '/home/ben/Dissertation/Multitask-Learning-With-GANs/5_mtl_wgan/tf-log'

generator_dims = [128 * gen_dim, 64 * gen_dim // 2, 64 * gen_dim // 4, 3]
discriminator_dims = [3, 64, 64 * 2, 64 * 4, 64 * 8, 1]

In [4]:
main_data = CIFAR100DataProvider(which_set='train', batch_size=batch_size)
aux_data = CIFAR100DataProvider(which_set='train', batch_size=batch_size, use_coarse_targets=True)
main_valid = CIFAR100DataProvider(which_set='valid', batch_size=batch_size)
aux_valid = CIFAR100DataProvider(which_set='valid', batch_size=batch_size, use_coarse_targets=True)
y_dim_main = 100
y_dim_aux = 20

In [5]:
_lmbda = 0.
num_shared_l_gen = 2
num_shared__l_disc = 2

In [6]:
model = MTLGAN(z_dim, batch_size, main_data, aux_data, main_valid, aux_valid,
               main_image_size, aux_image_size,
               y_dim_main, y_dim_aux, critic_iterations=5, lmbda=10)

model.create_networks(generator_dims, discriminator_dims, model_name,
                     num_shared_l_gen, num_shared__l_disc, _lmbda, optimizer,
                     gen_learning_rate, disc_learning_rate, optimizer_param)
model.initialize_networks(logs_dir)

model.train_model(max_itrs)

Setting up model...
Initializing network...
Training MTL-GAN model...
Evaluating summaries...
Time to go... 40.3718154483 mins
Step: 50/1000Saving images... Step: 50
Evaluating summaries...
Time to go... 10.6305958271 mins
Evaluating summaries...
Time to go... 9.95735850996 mins
Step: 100/1000Saving images... Step: 100
Evaluating summaries...
Time to go... 9.7169380188 mins
Evaluating summaries...
Time to go... 9.34698288308 mins
Saving images... Step: 150
Evaluating summaries...
Time to go... 9.13207535744 mins
Step: 200/1000Saving images... Step: 200
Evaluating summaries...
Time to go... 8.81507456038 mins
Evaluating summaries...
Time to go... 8.38049852053 mins
Step: 250/1000Saving images... Step: 250
Evaluating summaries...
Time to go... 8.07015480598 mins
Evaluating summaries...
Time to go... 7.61212515831 mins
Saving images... Step: 300
Step: 309/1000Ending Training...


In [12]:

model.main_generator_variables

[<tf.Variable 'main_generator/W_z:0' shape=(200, 32768) dtype=float32_ref>,
 <tf.Variable 'main_generator/gen_bnz/beta:0' shape=(2048,) dtype=float32_ref>,
 <tf.Variable 'main_generator/W_0:0' shape=(4, 4, 512, 2048) dtype=float32_ref>,
 <tf.Variable 'main_generator/b_0:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'main_generator/gen_bn0/beta:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'main_generator/W_1:0' shape=(4, 4, 256, 512) dtype=float32_ref>,
 <tf.Variable 'main_generator/b_1:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'main_generator/gen_bn1/beta:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'main_generator/W_pred:0' shape=(4, 4, 3, 256) dtype=float32_ref>,
 <tf.Variable 'main_generator/b_pred:0' shape=(3,) dtype=float32_ref>]

In [13]:
model.main_discriminator_variables

[<tf.Variable 'main_discriminator/W_0:0' shape=(4, 4, 3, 64) dtype=float32_ref>,
 <tf.Variable 'main_discriminator/b_0:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'main_discriminator/W_1:0' shape=(4, 4, 64, 128) dtype=float32_ref>,
 <tf.Variable 'main_discriminator/b_1:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'main_discriminator/W_2:0' shape=(4, 4, 128, 256) dtype=float32_ref>,
 <tf.Variable 'main_discriminator/b_2:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'main_discriminator/W_3:0' shape=(4, 4, 256, 512) dtype=float32_ref>,
 <tf.Variable 'main_discriminator/b_3:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'main_discriminator/W_pred:0' shape=(4, 4, 512, 1) dtype=float32_ref>,
 <tf.Variable 'main_discriminator/b_pred:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'main_discriminator/W_pred_with_y:0' shape=(101, 64) dtype=float32_ref>,
 <tf.Variable 'main_discriminator/b_pred_with_y:0' shape=(64,) dtype=float32_ref>]

In [14]:
model.aux_discriminator_variables

[<tf.Variable 'aux_discriminator/W_0:0' shape=(4, 4, 3, 64) dtype=float32_ref>,
 <tf.Variable 'aux_discriminator/b_0:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'aux_discriminator/W_1:0' shape=(4, 4, 64, 128) dtype=float32_ref>,
 <tf.Variable 'aux_discriminator/b_1:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'aux_discriminator/W_2:0' shape=(4, 4, 128, 256) dtype=float32_ref>,
 <tf.Variable 'aux_discriminator/b_2:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'aux_discriminator/W_3:0' shape=(4, 4, 256, 512) dtype=float32_ref>,
 <tf.Variable 'aux_discriminator/b_3:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'aux_discriminator/W_pred:0' shape=(4, 4, 512, 1) dtype=float32_ref>,
 <tf.Variable 'aux_discriminator/b_pred:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'aux_discriminator/W_pred_with_y:0' shape=(21, 64) dtype=float32_ref>,
 <tf.Variable 'aux_discriminator/b_pred_with_y:0' shape=(64,) dtype=float32_ref>]

In [15]:
model.aux_generator_variables

[<tf.Variable 'aux_generator/W_z:0' shape=(200, 32768) dtype=float32_ref>,
 <tf.Variable 'aux_generator/gen_bnz/beta:0' shape=(2048,) dtype=float32_ref>,
 <tf.Variable 'aux_generator/W_0:0' shape=(4, 4, 512, 2048) dtype=float32_ref>,
 <tf.Variable 'aux_generator/b_0:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'aux_generator/gen_bn0/beta:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'aux_generator/W_1:0' shape=(4, 4, 256, 512) dtype=float32_ref>,
 <tf.Variable 'aux_generator/b_1:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'aux_generator/gen_bn1/beta:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'aux_generator/W_pred:0' shape=(4, 4, 3, 256) dtype=float32_ref>,
 <tf.Variable 'aux_generator/b_pred:0' shape=(3,) dtype=float32_ref>]

In [7]:
def class_to_vec(class_name):
    map_list = list(map.copy())
    item_to_visualise = class_name
    indx = map_list.index(item_to_visualise)
    class_v = np.zeros([len(map)])
    class_v[indx] = 1
    class_v = [class_v for i in range(batch_size)]
    return np.array(class_v)

In [8]:
map = train_data.label_map
map

NameError: name 'train_data' is not defined

In [ ]:
print("Sampling images from model...")
class_v = class_to_vec("sunflower")
z = np.random.uniform(-1.0, 1.0, size=[batch_size, 200 - class_v.shape[1]]).astype(np.float32)
z = np.concatenate([z, class_v.astype(np.float32)], 1)

feed_dict = {model.z_vec: z, model.train_phase: True}

images = model.sess.run(model.gen_images, feed_dict=feed_dict)
shape = [1, 1]
utils.save_imshow_grid(images, model.logs_dir, "generated_sample_" + str(time.time()) + ".png", shape=shape)

In [ ]:
model.generate_sample(np.array(class_v))

In [ ]:
model.gen_images[0]